<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_15/S15_pl_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Install packages

In [ ]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git -C ERA_V1 pull
#!cd ../
!git clone "https://github.com/jyanivaddi/dl_hub.git"
!git -C dl_hub pull
!git pull
#!cd ../

!pip install --quiet "torchinfo" "seaborn" "pytorch-lightning" "torchmetrics" "lightning-bolts", "torchtext", "datasets", "tokenizers"
!pip install --quiet "prettytable"
!pip install --quiet "torchsummary"
!pip install --quiet "torch_lr_finder"
!pip install --quiet "grad-cam"
!pip install --quiet "gradio"

In [ ]:
import sys
sys.path.append("ERA_V1/session_15")
sys.path.append("dl_hub")

In [ ]:
!git -C dl_hub pull


remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 8 (delta 6), reused 8 (delta 6), pack-reused 0
Unpacking objects: 100% (8/8), 707 bytes | 353.00 KiB/s, done.
From https://github.com/jyanivaddi/dl_hub
   134449a..bbbcc6f  main       -> origin/main
Updating 134449a..bbbcc6f
Fast-forward
 Transformer/PL_model.py | 3 ++-
 Transformer/config.py   | 5 +++--
 2 files changed, 5 insertions(+), 3 deletions(-)


In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR
#from dl_hub.Transformer.config import get_config
from dl_hub.Transformer.config import get_config
from dl_hub.Transformer.PL_data_module import OpusDataModule
from dl_hub.Transformer.PL_model import LitTransformer
from dl_hub.Transformer.PL_main import train_transformer



In [ ]:
cfg = get_config()
cfg['batch_size'] = 16
cfg['d_model'] = 512
cfg['preload'] = None
cfg['num_epochs'] = 30
cfg["ds_mode"] = 'disk'
cfg["ds_path"] = "/home/e183534/ERA_Datasets/OpusBooks_En_It"


**Define data module**

In [ ]:
opus_dm = OpusDataModule(cfg)
opus_dm.prepare_data()
opus_dm.setup()
train_data_loader = opus_dm.train_dataloader()

**Define PL Model**

In [ ]:
tokenizer_src = opus_dm.tokenizer_src
tokenizer_tgt = opus_dm.tokenizer_tgt
loss_criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
transformer_model = LitTransformer(cfg, loss_criterion, tokenizer_src, tokenizer_tgt)
optimizer = torch.optim.Adam(transformer_model.model.parameters(), lr=cfg['lr'], eps=1e-9)
transformer_model.set_optimizer(optimizer)

scheduler = OneCycleLR(
    optimizer,
    max_lr=1E-3,
    steps_per_epoch=len(train_data_loader),
    epochs=cfg['num_epochs'],
    pct_start=5/cfg['num_epochs'],
    div_factor=10,
    three_phase=True,
    final_div_factor=10,
    anneal_strategy='linear'
)
transformer_model.set_scheduler_dict(scheduler, freq = 'step')

In [ ]:
train_transformer(transformer_model, opus_dm, cfg, epochs=cfg['num_epochs'])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | loss_criterion | CrossEntropyLoss | 0     
1 | model          | Transformer      | 75.1 M
----------------------------------------------------
75.1 M    Trainable params
0         Non-trainable params
75.1 M    Total params
300.532   Total estimated model params size (MB)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/e183534/ERA_V1/venv

Training: 0it [00:00, ?it/s]

/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.29' not found (required by /home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.28' not found (required by /home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (requ